In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
··········


In [1]:
!mkdir drive
!google-drive-ocamlfuse drive
!ls drive/Deep_learning_project

mkdir: cannot create directory ‘drive’: File exists
fuse: mountpoint is not empty
fuse: if you are sure this is safe, use the 'nonempty' mount option
 cache.py
 cifar-10-batches-py
 cifar10.py
'CIFAR10 with logistic regression 4.0.ipynb'
 CNN_Scratch.ipynb
 data
'Data Exploration and Preprocessing.ipynb'
 dataset.py
 data_utility.py
 download.py
 drive
 GR5242_Final_Project_Report.ipynb
 model
 ngrok
 ngrok-stable-linux-amd64.zip
 ngrok-stable-linux-amd64.zip.1
 __pycache__
 Untitled0.ipynb
'VGG_14&16.ipynb'
 VGG_19.ipynb
 vgg_logs


In [2]:
cd /content/drive/Deep_learning_project

/content/drive/Deep_learning_project


In [27]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from sklearn.metrics import confusion_matrix
import time
from datetime import timedelta
import math
import os

import cifar10

cifar10.maybe_download_and_extract()

images_train, cls_train, labels_train = cifar10.load_training_data()
images_test, cls_test, labels_test = cifar10.load_test_data()

#extract mean
mean = np.mean(images_train,axis=(0,1,2,3))
images_train=images_train-mean
images_test=images_test-mean

X_val=images_train[:500,:,:,:]
y_onehot_val=labels_train[:500,:]
X_train=images_train[500:,:,:,:]
y_onehot=labels_train[500:,:]

Data has apparently already been downloaded and unpacked.
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_1
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_2
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_3
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_4
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_5
Loading data: data/CIFAR-10/cifar-10-batches-py/test_batch


In [0]:
# evaluate performance on some data 
def perf_eval(logit_pred, y_true):
    """a function to evaluate performance of predicted y values vs true class labels"""
    # now look at some data
    print('    sample pred: {0}\n    sample true: {1}'.format(np.argmax(logit_pred[0:20],1),np.argmax(y_true[0:20],1)))
    # avg accuracy
    is_correct_vals = np.equal(np.argmax(logit_pred,1),np.argmax(y_true,1))
    #accuracy_vals = np.mean(is_correct_vals)
    #print('    mean classification accuracy: {0}%'.format(100*accuracy_vals))
    # Dig in a little deeper.  Where did we make correct predictions?  Does this seem reasonable?
    print('    correct predictions by class: {0}'.format(y_true[is_correct_vals,:].sum(axis=0)))
    
# cnn conv stuff
def conv(x, W):
    """simple wrapper for tf.nn.conv2d"""
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def maxpool(x):
    """simple wrapper for tf.nn.max_pool with stride size 2"""
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

def compute_cross_entropy(logits, y):
    # Compute the average cross-entropy across all the examples.
    numerical_instability_example = 0
    if numerical_instability_example:
        y_pred = tf.nn.softmax(logits, name='y_pred') # the predicted probability for each example.
        cross_ent = tf.reduce_mean(-tf.reduce_sum(y * tf.log(y_pred), reduction_indices=[1]))
    else:
        print(logits,y)
        sm_ce = tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=logits, name='cross_ent_terms')
        cross_ent = tf.reduce_mean(sm_ce, name='cross_ent')
    return cross_ent

def compute_accuracy(logits, y):
    prediction = tf.argmax(logits, 1, name='pred_class')
    true_label = tf.argmax(y, 1, name='true_class')
    accuracy = tf.reduce_mean(tf.cast(tf.equal(prediction, true_label), tf.float32))
    return accuracy

def test_model(opt_method,model_type,batch_size,iterations):
    start_time = time.time()
    dir_name = './logs/VGG/'
    n=50000
    n_val=500
    
    with tf.Graph().as_default():
    # We build the model here as before
        x = tf.placeholder(tf.float32, [None, 32*32*3], name='x')
        y = tf.placeholder(tf.float32, [None, 10], name='y')
        pkeep = tf.placeholder(tf.float32, name='pkeep')

        with tf.name_scope('model'):
            logits,weights = compute_logits(x, model_type, pkeep)
        with tf.name_scope('loss'):
            loss = compute_cross_entropy(logits=logits, y=y)
            regularizers=tf.nn.l2_loss(weights[0])
            for i in range(1,len(weights)):
                regularizers += tf.nn.l2_loss(weights[i])
            loss = tf.reduce_mean(loss + 0.0005 * regularizers)
        with tf.name_scope('accuracy'):
            accuracy = compute_accuracy(logits, y)

        with tf.name_scope('opt'):
            if opt_method == 'sgd':
                opt = tf.train.GradientDescentOptimizer(0.5)
            elif opt_method == 'rms':
                opt = tf.train.RMSPropOptimizer(.001)
            elif opt_method == 'adam':
                opt = tf.train.AdamOptimizer(1e-4)
            train_step = opt.minimize(loss)

        with tf.Session() as sess:
            summary_writer = tf.summary.FileWriter(dir_name, sess.graph)
            summary_writer_train = tf.summary.FileWriter(dir_name+'/train', sess.graph)
            summary_writer_val = tf.summary.FileWriter(dir_name+'/val')

            sess.run(tf.global_variables_initializer())
            
            train_ac=[]
            val_ac=[]
            
            for i in range(iterations):
                batch = np.floor(np.random.rand(batch_size)*(n-n_val)).astype(int)
                X_batch = X_train[batch,:,:,:].reshape([batch_size,-1])
                y_batch = y_onehot[batch]

                # now run
                _  = sess.run((train_step),feed_dict={x: X_batch, y: y_batch, pkeep:0.85})

                # write the summary output to file
                if i%100==0:
                    train_summary = tf.Summary(value=[tf.Summary.Value(tag="train_loss", simple_value=train_loss), 
                                          tf.Summary.Value(tag="train_accuracy", simple_value=train_acc)])
                    summary_writer_train.add_summary(summary, i)

                # print diagnostics
                if i%50 == 0:
                    X_batch = X_train[0:500,:,:,:].reshape([500,-1])
                    y_batch = y_onehot[0:500]
                    (train_error,train_logits) = sess.run((accuracy,logits), {x: X_batch, y: y_batch, pkeep:1.0})
                    print("\rStep {0:3d}: training accuracy {1:0.4f}".format(i, train_error), flush=True)
                    # further diagnostics
                    perf_eval(train_logits, y_batch)

                if i%50 == 0:
                    X_batch = X_val.reshape([n_val,-1])
                    y_batch = y_onehot_val
                    _,val_error,summary = sess.run((train_step,accuracy,summary_op), {x:X_batch, y:y_batch, pkeep:1.0})
                    print("\rStep {0:3d}: val accuracy {1:0.4f}".format(i, val_error), flush=True)
                    
                    train_ac.append(train_error)
                    val_ac.append(val_error)
                    summary_writer_val.add_summary(summary, i)
    
    end_time = time.time()
    time_dif = end_time - start_time
    print (val_ac,train_ac)
    print (time_dif/60)
    return val_ac,train_ac,time_dif/60

In [0]:
def compute_logits(x, model_type, pkeep):
    if model_type=='vgg3':
        x = tf.reshape(x, [-1,32,32,3])
        #cnn layer 1
        W_conv1 = tf.get_variable('W_conv1', shape=[3, 3, 3, 32])
        b_conv1 = tf.get_variable('b_conv1', shape=[32])
        h_conv1 = tf.nn.relu(tf.add(conv(x, W_conv1), b_conv1))
        #pool 1
        pool1=maxpool(h_conv1)
        #cnn layer 2
        W_conv2 = tf.get_variable('W_conv2', shape=[3, 3, 32, 64])
        b_conv2 = tf.get_variable('b_conv2', shape=[64])
        h_conv2 = tf.nn.relu(tf.add(conv(pool1, W_conv2), b_conv2))
        #pool 2
        pool2=maxpool(h_conv2)
        #cnn layer 3
        W_conv3 = tf.get_variable('W_conv3', shape=[3, 3, 64, 128])
        b_conv3 = tf.get_variable('b_conv3', shape=[128])
        h_conv3 = tf.nn.relu(tf.add(conv(pool2, W_conv3), b_conv3))
        #pool 3
        pool3=maxpool(h_conv3)
        # fc layer to logits
        shape=int(np.prod(pool3.get_shape()[1:]))
        flat = tf.reshape(pool3, [-1, shape])
        W_fc1 = tf.get_variable('W_fc1', [shape, 10])
        b_fc1 = tf.get_variable('b_fc1', [10])
        logits = tf.add(tf.matmul(flat, W_fc1), b_fc1, name='logits_vgg4_small_filters')
        weights=[W_conv1,W_conv2]
    elif model_type=='vgg14':
        x = tf.reshape(x, [-1,32,32,3])
        #cnn layer 1
        W_conv1 = tf.get_variable('W_conv1', shape=[3, 3, 3, 64])
        b_conv1 = tf.get_variable('b_conv1', shape=[64])
        h_conv1 = tf.nn.relu(tf.add(conv(x, W_conv1), b_conv1))
        #cnn layer 2
        W_conv2 = tf.get_variable('W_conv2', shape=[3, 3, 64, 64])
        b_conv2 = tf.get_variable('b_conv2', shape=[64])
        h_conv2 = tf.nn.relu(tf.add(conv(h_conv1, W_conv2), b_conv2))
        #pool 1
        pool1=maxpool(h_conv2)
        #cnn layer 3
        W_conv3 = tf.get_variable('W_conv3', shape=[3, 3, 64, 128])
        b_conv3 = tf.get_variable('b_conv3', shape=[128])
        h_conv3 = tf.nn.relu(tf.add(conv(pool1, W_conv3), b_conv3))
        #cnn layer 4
        W_conv4 = tf.get_variable('W_conv4', shape=[3, 3, 128, 128])
        b_conv4 = tf.get_variable('b_conv4', shape=[128])
        h_conv4 = tf.nn.relu(tf.add(conv(h_conv3, W_conv4), b_conv4))
        #pool 2
        pool2=maxpool(h_conv4)
        #cnn layer 5
        W_conv5 = tf.get_variable('W_conv5', shape=[3, 3, 128, 256])
        b_conv5 = tf.get_variable('b_conv5', shape=[256])
        h_conv5 = tf.nn.relu(tf.add(conv(pool2, W_conv5), b_conv5))
        #cnn layer 6
        W_conv6 = tf.get_variable('W_conv6', shape=[3, 3, 256, 256])
        b_conv6 = tf.get_variable('b_conv6', shape=[256])
        h_conv6 = tf.nn.relu(tf.add(conv(h_conv5, W_conv6), b_conv6))
        #cnn layer 7
        W_conv7 = tf.get_variable('W_conv7', shape=[3, 3, 256, 256])
        b_conv7 = tf.get_variable('b_conv7', shape=[256])
        h_conv7 = tf.nn.relu(tf.add(conv(h_conv6, W_conv7), b_conv7))
        #pool 3
        pool3=maxpool(h_conv7)
        #cnn layer 8
        W_conv8 = tf.get_variable('W_conv8', shape=[3, 3, 256, 512])
        b_conv8 = tf.get_variable('b_conv8', shape=[512])
        h_conv8 = tf.nn.relu(tf.add(conv(pool3, W_conv8), b_conv8))
        #cnn layer 9
        W_conv9 = tf.get_variable('W_conv9', shape=[3, 3, 512, 512])
        b_conv9 = tf.get_variable('b_conv9', shape=[512])
        h_conv9 = tf.nn.relu(tf.add(conv(h_conv8, W_conv9), b_conv9))
        #cnn layer 10
        W_conv10 = tf.get_variable('W_conv10', shape=[3, 3, 512, 512])
        b_conv10 = tf.get_variable('b_conv10', shape=[512])
        h_conv10 = tf.nn.relu(tf.add(conv(h_conv9, W_conv10), b_conv10))
        #pool 4
        pool4=maxpool(h_conv10)
        #cnn layer 11
        W_conv11 = tf.get_variable('W_conv11', shape=[3, 3, 512, 512])
        b_conv11 = tf.get_variable('b_conv11', shape=[512])
        h_conv11 = tf.nn.relu(tf.add(conv(pool4, W_conv11), b_conv11))
        #cnn layer 12
        W_conv12 = tf.get_variable('W_conv12', shape=[3, 3, 512, 512])
        b_conv12 = tf.get_variable('b_conv12', shape=[512])
        h_conv12 = tf.nn.relu(tf.add(conv(h_conv11, W_conv12), b_conv12))
        #cnn layer 13
        W_conv13 = tf.get_variable('W_conv13', shape=[3, 3, 512, 512])
        b_conv13 = tf.get_variable('b_conv13', shape=[512])
        h_conv13 = tf.nn.relu(tf.add(conv(h_conv12, W_conv13), b_conv13))
        #pool 5
        pool5=maxpool(h_conv13)
        #fc1-4096
        shape=int(np.prod(pool5.get_shape()[1:]))
        pool5_flat=tf.reshape(pool5,[-1,shape])
        W_fc1 = tf.get_variable('W_fc1', shape=[shape,4096])
        b_fc1 = tf.Variable(tf.constant(1.0, shape=[4096], dtype=tf.float32),name='b_fc1')
        h_fc1 = tf.nn.relu(tf.add(tf.matmul(pool5_flat, W_fc1), b_fc1))
        # fc layer to logits
        W_fc = tf.get_variable('W_fc', [4096, 10])
        b_fc = tf.get_variable('b_fc', [10])
        logits = tf.add(tf.matmul(h_fc1, W_fc), b_fc, name='logits_vgg13')
        weights=[W_conv11,W_conv12,W_conv13,W_fc1]
        #,W_conv3,W_conv4,W_conv5,W_conv6,W_conv7,W_conv
    elif model_type=='vgg16':
        x = tf.reshape(x, [-1,32,32,3])
        #cnn layer 1
        W_conv1 = tf.get_variable('W_conv1', shape=[3, 3, 3, 64])
        b_conv1 = tf.get_variable('b_conv1', shape=[64])
        h_conv1 = tf.nn.relu(tf.add(conv(x, W_conv1), b_conv1))
        #cnn layer 2
        W_conv2 = tf.get_variable('W_conv2', shape=[3, 3, 64, 64])
        b_conv2 = tf.get_variable('b_conv2', shape=[64])
        h_conv2 = tf.nn.relu(tf.add(conv(h_conv1, W_conv2), b_conv2))
        #pool 1
        pool1=maxpool(h_conv2)
        #cnn layer 3
        W_conv3 = tf.get_variable('W_conv3', shape=[3, 3, 64, 128])
        b_conv3 = tf.get_variable('b_conv3', shape=[128])
        h_conv3 = tf.nn.relu(tf.add(conv(pool1, W_conv3), b_conv3))
        #cnn layer 4
        W_conv4 = tf.get_variable('W_conv4', shape=[3, 3, 128, 128])
        b_conv4 = tf.get_variable('b_conv4', shape=[128])
        h_conv4 = tf.nn.relu(tf.add(conv(h_conv3, W_conv4), b_conv4))
        #pool 2
        pool2=maxpool(h_conv4)
        #cnn layer 5
        W_conv5 = tf.get_variable('W_conv5', shape=[3, 3, 128, 256])
        b_conv5 = tf.get_variable('b_conv5', shape=[256])
        h_conv5 = tf.nn.relu(tf.add(conv(pool2, W_conv5), b_conv5))
        #cnn layer 6
        W_conv6 = tf.get_variable('W_conv6', shape=[3, 3, 256, 256])
        b_conv6 = tf.get_variable('b_conv6', shape=[256])
        h_conv6 = tf.nn.relu(tf.add(conv(h_conv5, W_conv6), b_conv6))
        #cnn layer 7
        W_conv7 = tf.get_variable('W_conv7', shape=[3, 3, 256, 256])
        b_conv7 = tf.get_variable('b_conv7', shape=[256])
        h_conv7 = tf.nn.relu(tf.add(conv(h_conv6, W_conv7), b_conv7))
        #pool 3
        pool3=maxpool(h_conv7)
        #cnn layer 8
        W_conv8 = tf.get_variable('W_conv8', shape=[3, 3, 256, 512])
        b_conv8 = tf.get_variable('b_conv8', shape=[512])
        h_conv8 = tf.nn.relu(tf.add(conv(pool3, W_conv8), b_conv8))
        #cnn layer 9
        W_conv9 = tf.get_variable('W_conv9', shape=[3, 3, 512, 512])
        b_conv9 = tf.get_variable('b_conv9', shape=[512])
        h_conv9 = tf.nn.relu(tf.add(conv(h_conv8, W_conv9), b_conv9))
        #cnn layer 10
        W_conv10 = tf.get_variable('W_conv10', shape=[3, 3, 512, 512])
        b_conv10 = tf.get_variable('b_conv10', shape=[512])
        h_conv10 = tf.nn.relu(tf.add(conv(h_conv9, W_conv10), b_conv10))
        #pool 4
        pool4=maxpool(h_conv10)
        #cnn layer 11
        W_conv11 = tf.get_variable('W_conv11', shape=[3, 3, 512, 512])
        b_conv11 = tf.get_variable('b_conv11', shape=[512])
        h_conv11 = tf.nn.relu(tf.add(conv(pool4, W_conv11), b_conv11))
        #cnn layer 12
        W_conv12 = tf.get_variable('W_conv12', shape=[3, 3, 512, 512])
        b_conv12 = tf.get_variable('b_conv12', shape=[512])
        h_conv12 = tf.nn.relu(tf.add(conv(h_conv11, W_conv12), b_conv12))
        #cnn layer 13
        W_conv13 = tf.get_variable('W_conv13', shape=[3, 3, 512, 512])
        b_conv13 = tf.get_variable('b_conv13', shape=[512])
        h_conv13 = tf.nn.relu(tf.add(conv(h_conv12, W_conv13), b_conv13))
        #pool 5
        pool5=maxpool(h_conv13)
        #fc1-4096
        shape=int(np.prod(pool5.get_shape()[1:]))
        pool5_flat=tf.reshape(pool5,[-1,shape])
        W_fc1 = tf.get_variable('W_fc1', shape=[shape,4096])
        b_fc1 = tf.Variable(tf.constant(1.0, shape=[4096], dtype=tf.float32),name='b_fc1')
        h_fc1 = tf.nn.relu(tf.add(tf.matmul(pool5_flat, W_fc1), b_fc1))
        #fc2-4096
        W_fc2 = tf.get_variable('W_fc2', shape=[4096,4096])
        b_fc2 = tf.Variable(tf.constant(1.0, shape=[4096], dtype=tf.float32),name='b_fc2')
        h_fc2 = tf.nn.relu(tf.add(tf.matmul(h_fc1, W_fc2), b_fc2))
        #fc3-1000
        W_fc3 = tf.get_variable('W_fc3', shape=[4096,1000])
        b_fc3 = tf.Variable(tf.constant(1.0, shape=[1000], dtype=tf.float32),name='b_fc3')
        h_fc3 = tf.nn.relu(tf.add(tf.matmul(h_fc2, W_fc3), b_fc3))
        # fc layer to logits
        W_fc = tf.get_variable('W_fc', [1000, 10])
        b_fc = tf.get_variable('b_fc', [10])
        logits = tf.add(tf.matmul(h_fc3, W_fc), b_fc, name='logits_vgg13')
        weights=[W_conv6,W_conv8,W_conv10,W_conv11,W_conv12,W_conv13,W_fc1,W_fc2]
    return logits,weights

In [7]:
accuracy1_val,accuracy1_train,time1=test_model('adam','vgg14',64,8001)

Tensor("model/logits_vgg13:0", shape=(?, 10), dtype=float32) Tensor("y:0", shape=(?, 10), dtype=float32)
Step   0: training accuracy 0.1120
    sample pred: [2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [ 0.  0. 56.  0.  0.  0.  0.  0.  0.  0.]
Step   0: val accuracy 0.0860
Step  50: training accuracy 0.1080
    sample pred: [8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [ 0.  0.  0.  0.  0.  0.  0.  0. 54.  0.]
Step  50: val accuracy 0.0900
Step 100: training accuracy 0.0900
    sample pred: [5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [ 0.  0.  0.  0.  0. 45.  0.  0.  0.  0.]
Step 100: val accuracy 0.0800
Step 150: training accuracy 0.0940
    sample pred: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
    sample true: [5 8 2 

In [8]:
print(accuracy1_val)

[0.086, 0.09, 0.08, 0.13, 0.158, 0.192, 0.216, 0.232, 0.25, 0.254, 0.328, 0.29, 0.324, 0.314, 0.348, 0.336, 0.364, 0.352, 0.36, 0.348, 0.336, 0.376, 0.338, 0.396, 0.438, 0.4, 0.418, 0.462, 0.446, 0.432, 0.47, 0.47, 0.49, 0.48, 0.496, 0.494, 0.498, 0.498, 0.516, 0.53, 0.532, 0.498, 0.57, 0.542, 0.526, 0.556, 0.596, 0.592, 0.562, 0.518, 0.592, 0.594, 0.596, 0.594, 0.562, 0.598, 0.602, 0.616, 0.614, 0.604, 0.61, 0.618, 0.634, 0.606, 0.582, 0.632, 0.634, 0.626, 0.63, 0.632, 0.64, 0.646, 0.61, 0.664, 0.65, 0.664, 0.672, 0.648, 0.68, 0.664, 0.684, 0.668, 0.67, 0.682, 0.662, 0.674, 0.642, 0.646, 0.664, 0.692, 0.666, 0.67, 0.664, 0.682, 0.688, 0.704, 0.694, 0.67, 0.706, 0.694, 0.692, 0.712, 0.706, 0.694, 0.694, 0.712, 0.72, 0.71, 0.688, 0.686, 0.708, 0.718, 0.702, 0.716, 0.718, 0.732, 0.708, 0.728, 0.726, 0.752, 0.742, 0.72, 0.756, 0.724, 0.698, 0.744, 0.722, 0.742, 0.728, 0.732, 0.74, 0.742, 0.752, 0.718, 0.73, 0.748, 0.754, 0.746, 0.752, 0.762, 0.77, 0.738, 0.752, 0.74, 0.738, 0.766, 0.73, 0

In [9]:
print(accuracy1_train)

[0.112, 0.108, 0.09, 0.094, 0.132, 0.202, 0.194, 0.254, 0.232, 0.224, 0.278, 0.294, 0.328, 0.296, 0.33, 0.292, 0.32, 0.34, 0.318, 0.328, 0.344, 0.376, 0.32, 0.386, 0.388, 0.376, 0.408, 0.42, 0.392, 0.414, 0.39, 0.446, 0.486, 0.46, 0.474, 0.444, 0.488, 0.506, 0.492, 0.498, 0.512, 0.486, 0.542, 0.52, 0.564, 0.52, 0.58, 0.558, 0.566, 0.538, 0.59, 0.596, 0.562, 0.576, 0.592, 0.61, 0.594, 0.634, 0.636, 0.62, 0.62, 0.618, 0.656, 0.616, 0.586, 0.62, 0.634, 0.66, 0.67, 0.648, 0.664, 0.66, 0.608, 0.678, 0.68, 0.674, 0.666, 0.66, 0.694, 0.676, 0.714, 0.692, 0.694, 0.718, 0.704, 0.712, 0.686, 0.692, 0.698, 0.702, 0.718, 0.71, 0.72, 0.724, 0.732, 0.756, 0.712, 0.714, 0.764, 0.752, 0.75, 0.75, 0.742, 0.79, 0.746, 0.784, 0.776, 0.776, 0.76, 0.774, 0.776, 0.786, 0.792, 0.78, 0.79, 0.8, 0.808, 0.794, 0.79, 0.782, 0.796, 0.83, 0.806, 0.788, 0.796, 0.816, 0.832, 0.816, 0.794, 0.8, 0.808, 0.806, 0.826, 0.772, 0.826, 0.806, 0.838, 0.842, 0.852, 0.856, 0.874, 0.83, 0.82, 0.878, 0.856, 0.854, 0.874, 0.848, 

In [10]:
accuracy2_val,accuracy2_train,time2=test_model('adam','vgg14',64,20001)

Tensor("model/logits_vgg13:0", shape=(?, 10), dtype=float32) Tensor("y:0", shape=(?, 10), dtype=float32)
Step   0: training accuracy 0.0900
    sample pred: [5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [ 0.  0.  0.  0.  0. 45.  0.  0.  0.  0.]
Step   0: val accuracy 0.0800
Step  50: training accuracy 0.0980
    sample pred: [4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [ 0.  0.  0.  0. 49.  0.  0.  0.  0.  0.]
Step  50: val accuracy 0.1000
Step 100: training accuracy 0.1080
    sample pred: [6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [ 0.  0.  0.  0.  0.  0. 54.  0.  0.  0.]
Step 100: val accuracy 0.1060
Step 150: training accuracy 0.0940
    sample pred: [9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9]
    sample true: [5 8 2 

In [11]:
print(accuracy2_val)

[0.08, 0.1, 0.106, 0.112, 0.18, 0.202, 0.21, 0.25, 0.266, 0.32, 0.308, 0.304, 0.288, 0.33, 0.316, 0.354, 0.402, 0.386, 0.398, 0.396, 0.44, 0.464, 0.426, 0.44, 0.456, 0.46, 0.486, 0.488, 0.502, 0.474, 0.484, 0.516, 0.538, 0.526, 0.528, 0.522, 0.516, 0.548, 0.53, 0.54, 0.562, 0.548, 0.542, 0.568, 0.552, 0.536, 0.55, 0.596, 0.596, 0.58, 0.592, 0.598, 0.574, 0.618, 0.604, 0.618, 0.628, 0.586, 0.638, 0.626, 0.626, 0.55, 0.646, 0.616, 0.656, 0.652, 0.666, 0.646, 0.662, 0.652, 0.652, 0.66, 0.672, 0.652, 0.614, 0.652, 0.65, 0.698, 0.676, 0.676, 0.662, 0.672, 0.658, 0.698, 0.7, 0.664, 0.68, 0.684, 0.682, 0.7, 0.706, 0.674, 0.696, 0.688, 0.712, 0.716, 0.736, 0.718, 0.7, 0.676, 0.7, 0.704, 0.716, 0.69, 0.706, 0.716, 0.684, 0.704, 0.708, 0.724, 0.68, 0.692, 0.72, 0.694, 0.722, 0.73, 0.716, 0.734, 0.738, 0.742, 0.732, 0.716, 0.732, 0.722, 0.74, 0.744, 0.74, 0.75, 0.75, 0.728, 0.706, 0.73, 0.742, 0.708, 0.724, 0.728, 0.742, 0.73, 0.716, 0.728, 0.732, 0.726, 0.746, 0.722, 0.73, 0.738, 0.738, 0.74, 0.

In [12]:
accuracy2_val,accuracy2_train,time2=test_model('adam','vgg16',64,20001)

Tensor("model/logits_vgg13:0", shape=(?, 10), dtype=float32) Tensor("y:0", shape=(?, 10), dtype=float32)
Step   0: training accuracy 0.1020
    sample pred: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [51.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
Step   0: val accuracy 0.1020
Step  50: training accuracy 0.1080
    sample pred: [8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [ 0.  0.  0.  0.  0.  0.  0.  0. 54.  0.]
Step  50: val accuracy 0.0900
Step 100: training accuracy 0.0900
    sample pred: [5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [ 0.  0.  0.  0.  0. 45.  0.  0.  0.  0.]
Step 100: val accuracy 0.0800
Step 150: training accuracy 0.1040
    sample pred: [7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7]
    sample true: [5 8 2 

# VGG16

In [13]:
accuracy2_val,accuracy2_train,time2=test_model('adam','vgg16',64,20001)

Tensor("model/logits_vgg13:0", shape=(?, 10), dtype=float32) Tensor("y:0", shape=(?, 10), dtype=float32)
Step   0: training accuracy 0.0940
    sample pred: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [ 0. 47.  0.  0.  0.  0.  0.  0.  0.  0.]
Step   0: val accuracy 0.1300
Step  50: training accuracy 0.0980
    sample pred: [4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [ 0.  0.  0.  0. 49.  0.  0.  0.  0.  0.]
Step  50: val accuracy 0.1000
Step 100: training accuracy 0.1080
    sample pred: [6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6]
    sample true: [5 8 2 8 0 4 1 8 9 8 2 9 9 2 7 5 7 3 8 8]
    correct predictions by class: [ 0.  0.  0.  0.  0.  0. 54.  0.  0.  0.]
Step 100: val accuracy 0.1060
Step 150: training accuracy 0.0900
    sample pred: [3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3]
    sample true: [5 8 2 

In [14]:
print(accuracy2_val)

[0.13, 0.1, 0.106, 0.094, 0.08, 0.094, 0.102, 0.106, 0.1, 0.1, 0.15, 0.184, 0.234, 0.256, 0.198, 0.178, 0.242, 0.278, 0.232, 0.28, 0.266, 0.286, 0.282, 0.318, 0.27, 0.37, 0.314, 0.336, 0.332, 0.384, 0.346, 0.4, 0.362, 0.378, 0.372, 0.392, 0.398, 0.372, 0.438, 0.374, 0.37, 0.406, 0.36, 0.44, 0.448, 0.442, 0.45, 0.414, 0.494, 0.438, 0.484, 0.47, 0.46, 0.466, 0.438, 0.508, 0.506, 0.468, 0.508, 0.494, 0.524, 0.528, 0.508, 0.466, 0.486, 0.506, 0.504, 0.522, 0.474, 0.516, 0.494, 0.5, 0.516, 0.534, 0.518, 0.548, 0.514, 0.5, 0.536, 0.554, 0.528, 0.544, 0.556, 0.55, 0.578, 0.538, 0.558, 0.56, 0.554, 0.558, 0.53, 0.548, 0.55, 0.53, 0.57, 0.57, 0.568, 0.564, 0.57, 0.558, 0.576, 0.554, 0.55, 0.544, 0.592, 0.582, 0.588, 0.556, 0.558, 0.558, 0.578, 0.578, 0.572, 0.588, 0.576, 0.584, 0.582, 0.592, 0.568, 0.598, 0.584, 0.572, 0.596, 0.608, 0.57, 0.578, 0.584, 0.582, 0.594, 0.604, 0.596, 0.608, 0.614, 0.642, 0.62, 0.59, 0.614, 0.572, 0.628, 0.606, 0.626, 0.614, 0.6, 0.604, 0.626, 0.628, 0.61, 0.628, 0.

In [0]:
# Run Tensorflow in the background - note that we specify the log 
# directory we want to look at
LOG_DIR = 'logs/VGG'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

In [24]:
# Download and unzip ngrok - you will only need to do this once per session
! wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
! unzip ngrok-stable-linux-amd64.zip

--2018-12-11 05:27:32--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 34.196.237.103, 34.232.40.183, 52.204.188.97, ...
Connecting to bin.equinox.io (bin.equinox.io)|34.196.237.103|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5363700 (5.1M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.3’

ngrok-stable-linux- 100%[===================>]   5.11M  4.26MB/s    in 1.2s    

2018-12-11 05:27:38 (4.26 MB/s) - ‘ngrok-stable-linux-amd64.zip.3’ saved [5363700/5363700]

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [0]:
# Launch the ngrok background process
get_ipython().system_raw('./ngrok http 6006 &')

In [26]:
# Get the public URL and be sorted!
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

http://161753ea.ngrok.io
